<h1>Remove Background with AI</h1>
Now we're finally going to start to work with AI-Models. In this project we're going to use the U-2-Net model (U Square Net) to remove the background of images.

<h3>Setup</h3>
We mount our drive and navigate to our Workshop Folder.


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
%cd /content/drive/MyDrive/AiXDesignKitchen/Workshop/Playground

/content/drive/MyDrive/AiXDesignKitchen/Workshop/Playground


<h3>Clone Model from Github</h3>
To be able to use the pretrained model we're cloning the repository from GitHub.

In [4]:
!git clone https://github.com/shreyas-bk/U-2-Net
%cd /content/drive/MyDrive/AiXDesignKitchen/Workshop/Playground/U-2-Net

fatal: destination path 'U-2-Net' already exists and is not an empty directory.
/content/drive/MyDrive/AiXDesignKitchen/Workshop/Playground/U-2-Net


We also create folders for image in- and output and assign variables to them.

In [5]:
!mkdir images
input_path = '/content/drive/MyDrive/AiXDesignKitchen/Workshop/Playground/U-2-Net/images/'
!mkdir results
output_path = '/content/drive/MyDrive/AiXDesignKitchen/Workshop/Playground/U-2-Net/results/'
!mkdir results/FinalOutput
final_path = '/content/drive/MyDrive/AiXDesignKitchen/Workshop/Playground/U-2-Net/results/FinalOutput/'

mkdir: cannot create directory ‘images’: File exists
mkdir: cannot create directory ‘results’: File exists
mkdir: cannot create directory ‘results/FinalOutput’: File exists


<h3>Import Modules</h3>
Then we import all necessary modules.

In [6]:
from google.colab import files
import os
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.preprocessing.image import img_to_array
import numpy as np
from PIL import Image as Img
import cv2
from google.colab.patches import cv2_imshow
print('Done!')

Done!


<h3>Upload Images</h3>


In [7]:
%cd '/content/drive/MyDrive/AiXDesignKitchen/Workshop/Playground/U-2-Net/images/'
uploaded = files.upload()
%cd .. 

/content/drive/MyDrive/AiXDesignKitchen/Workshop/Playground/U-2-Net/images


Saving Beyoncé.jpg to Beyoncé.jpg
/content/drive/MyDrive/AiXDesignKitchen/Workshop/Playground/U-2-Net


Config

In [8]:
!python -W ignore u2net_test.py
 
image_dir = os.path.join(os.getcwd(), input_path)
names = [name[:-4] for name in os.listdir(image_dir)]
THRESHOLD = 0.9
RESCALE = 255
LAYER = 2
COLOR = (0, 0, 0)
THICKNESS = 4
SAL_SHIFT = 100


['/content/drive/MyDrive/AiXDesignKitchen/Workshop/Playground/U-2-Net/images/Bllo.jpg', '/content/drive/MyDrive/AiXDesignKitchen/Workshop/Playground/U-2-Net/images/Bllo2.jpg', '/content/drive/MyDrive/AiXDesignKitchen/Workshop/Playground/U-2-Net/images/Merkel.jpg', '/content/drive/MyDrive/AiXDesignKitchen/Workshop/Playground/U-2-Net/images/Beyoncé.jpg']
inferencing: Bllo.jpg
inferencing: Bllo2.jpg
inferencing: Merkel.jpg
inferencing: Beyoncé.jpg


Render Results

In [9]:
for name in names:
 
  # BACKGROUND REMOVAL
 
  output = load_img(output_path+name+'.png')
  out_img = img_to_array(output)
  out_img /= RESCALE
 
  out_img[out_img > THRESHOLD] = 1
  out_img[out_img <= THRESHOLD] = 0
 
  shape = out_img.shape
  a_layer_init = np.ones(shape = (shape[0],shape[1],1))
  mul_layer = np.expand_dims(out_img[:,:,0],axis=2)
  a_layer = mul_layer*a_layer_init
  rgba_out = np.append(out_img,a_layer,axis=2)
 
  input = load_img(input_path+name+'.jpg')
  inp_img = img_to_array(input)
  inp_img /= RESCALE
 
  a_layer = np.ones(shape = (shape[0],shape[1],1))
  rgba_inp = np.append(inp_img,a_layer,axis=2)
 
  rem_back = (rgba_inp*rgba_out)
  rem_back_scaled = rem_back*RESCALE
 
  # BOUNDING BOX CREATION
 
  out_layer = out_img[:,:,LAYER]
  x_starts = [np.where(out_layer[i]==1)[0][0] if len(np.where(out_layer[i]==1)[0])!=0 else out_layer.shape[0]+1 for i in range(out_layer.shape[0])]
  x_ends = [np.where(out_layer[i]==1)[0][-1] if len(np.where(out_layer[i]==1)[0])!=0 else 0 for i in range(out_layer.shape[0])]
  y_starts = [np.where(out_layer.T[i]==1)[0][0] if len(np.where(out_layer.T[i]==1)[0])!=0 else out_layer.T.shape[0]+1 for i in range(out_layer.T.shape[0])]
  y_ends = [np.where(out_layer.T[i]==1)[0][-1] if len(np.where(out_layer.T[i]==1)[0])!=0 else 0 for i in range(out_layer.T.shape[0])]
  
  startx = min(x_starts)
  endx = max(x_ends)
  starty = min(y_starts)
  endy = max(y_ends)
  start = (startx,starty)
  end = (endx,endy)
 
  box_img = inp_img.copy()
  box_img = cv2.rectangle(box_img, start, end, COLOR, THICKNESS)
  box_img = np.append(box_img,a_layer,axis=2)
  box_img_scaled = box_img*RESCALE
 
  # SALIENT FEATURE MAP
 
  sal_img = inp_img.copy()
  add_layer = out_img.copy()
  add_layer[add_layer==1] = SAL_SHIFT/RESCALE
  sal_img[:,:,LAYER] += add_layer[:,:,LAYER]
  sal_img = np.append(sal_img,a_layer,axis=2)
  sal_img_scaled = sal_img*RESCALE
  sal_img_scaled[sal_img_scaled>RESCALE] = RESCALE
 
  # OUTPUT RESULTS
 
  inp_img*=RESCALE
  inp_img = np.append(inp_img,RESCALE*a_layer,axis=2)
  inp_img = cv2.resize(inp_img,(int(shape[1]/3),int(shape[0]/3)))
  rem_back = cv2.resize(rem_back_scaled,(int(shape[1]/3),int(shape[0]/3)))
  box_img = cv2.resize(box_img_scaled,(int(shape[1]/3),int(shape[0]/3)))
  sal_img = cv2.resize(sal_img_scaled,(int(shape[1]/3),int(shape[0]/3)))
  result = np.concatenate((inp_img,rem_back,box_img,sal_img),axis=1)
  result_img = Img.fromarray(result.astype('uint8'), 'RGBA')
  final_img = Img.fromarray(rem_back.astype('uint8'), 'RGBA')
  final_img.save(final_path+name+'_background_removed.png')
  print('Image has been saved to ' + final_path)
  print('\nINPUT                                    BACKGROUND REMOVED                     BOUNDING BOX                               SALIENT MAP\n')
  display(result_img)

Output hidden; open in https://colab.research.google.com to view.